In [25]:
import numpy as np
import matplotlib.pyplot as plt 

from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
from skimage.io import imread

In [31]:
import numpy as np

def read_mhd_file(mhd_filename):
    header = {}
    with open(mhd_filename, 'r') as f:
        for line in f:
            if '=' in line:
                key, value = line.strip().split('=')
                header[key.strip()] = value.strip()
    return header

def load_raw_data(header, mhd_dir='.'):
    # Get data type
    dtype_map = {
        'MET_CHAR': np.int8,
        'MET_UCHAR': np.uint8,
        'MET_SHORT': np.int16,
        'MET_USHORT': np.uint16,
        'MET_INT': np.int32,
        'MET_UINT': np.uint32,
        'MET_FLOAT': np.float32,
        'MET_DOUBLE': np.float64,
    }
    dtype = dtype_map[header['ElementType']] #GATE uses ElementType strings to describe how the voxel values are stored

    # Get dimensions
    dim_size = [int(x) for x in header['DimSize'].split()]

    # Get data file path
    raw_file = header['ElementDataFile']
    raw_file_path = f"{mhd_dir}/{raw_file}"

    # Read binary data
    data = np.fromfile(raw_file_path, dtype=dtype)
    data = data.reshape(dim_size[::-1])  # reverse order (z, y, x)

    return data

# Usage
mhd_file = 'output/true_activity_map_edep.mhd'
header = read_mhd_file(mhd_file)
activity_data = load_raw_data(header, mhd_dir='output')

print("Shape:", activity_data.shape)

#Integrating over z-axis (z, y, x)
activity_data = np.transpose(activity_data, (2, 1,0))
projection_image = np.sum(activity_data, axis=2)

Shape: (180, 180, 180)


In [34]:
# Plot it in a clean figure
fig = plt.figure(figsize=(6, 6), frameon=False)
ax = plt.Axes(fig, [0., 0., 1., 1.])  # Full canvas usage
ax.set_axis_off()
fig.add_axes(ax)
ax.imshow(projection_image.T, cmap='gray', origin='lower')
#plt.show()

# Save the image-only output
fig.savefig("dose_map.png", dpi=300)
plt.close(fig)

/var/folders/qv/434ty_nj1bdfm_6l1bc5ppg80000gn/T/ipykernel_68534/2973315267.py:10: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "facecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig.savefig("dose_map.png", dpi=300)
/var/folders/qv/434ty_nj1bdfm_6l1bc5ppg80000gn/T/ipykernel_68534/2973315267.py:10: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "edgecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig.savefig("dose_map.png", dpi=300)
/var/folders/qv/434ty_nj1bdfm_6l1bc5ppg80000gn/T/ipykernel_68534/2973315267.py:10: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "orientation" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig.savefig("dose_map.png", dpi=300)
/var/folders/qv/434ty_nj1bdfm_6l1bc5ppg80000gn/T/ipykernel_68534/2973315267.py:10: MatplotlibDeprecationWarning: 

In [ ]:
# Load and convert images to float
img_pet = img_as_float(imread("dose_map.png", as_gray=True))
img_philips = img_as_float(imread("fbp_reco.png", as_gray=True))

# Ensure same shape
assert img_pet.shape == img_philips.shape, "Images must be the same size."

# Compute metrics
mse_val = mean_squared_error(img_philips, img_pet)
ssim_val = ssim(img_philips, img_pet, data_range=img_philips.max() - img_philips.min())

# Plot images
fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(img_philips, cmap='gray', vmin=0, vmax=1)
ax[0].set_title('Philips Estimate')
ax[0].axis('off')

ax[1].imshow(img_pet, cmap='gray', vmin=0, vmax=1)
ax[1].set_title(f'PET Estimate\nMSE: {mse_val:.4f}, SSIM: {ssim_val:.4f}')
ax[1].axis('off')

plt.tight_layout()
plt.show()


ValueError: Input images must have the same dimensions.